In [ ]:
from jupyter_core.application import JupyterApp
from traitlets import Unicode
from traitlets.config import Configurable, Config

# Creating a Configurable class

In [ ]:
class A(Configurable):
    my_trait = Unicode("default").tag(config=True)
    
    def _repr_markdown_(self):
        return f"**{self.__class__.__name__}**.*my_trait*: {self.my_trait}"
A()

In [ ]:
config_dict = {
    "A":{
        "my_trait": "Not the default"    
    }
}

config = Config(config_dict)
A(config=config)

In [ ]:
A(my_trait="I set this in my `__init__` through \*\*kwargs.")

# Subclassing Configurable classes

In [ ]:
class B(A):
    pass

display(B())

## Subclasses inherit parent's settings

In [ ]:
display(config)
display(B(config=config))

## Specificity wins

In [ ]:
config.B.my_trait="B now has a new value"

display(config)
B(config=config)

## Configuration for classes that can't be Configurable

E.g., `notebook.base.handlers.APIHandler` will break if you multiply inherit `Configurable`

Solution:

- Create a `Configurable` class next to the nonconfigurable class
- Pass `Config` object into nonconfigurable class
- In the nonconfigurable class, instantiate the `Configurable` class with the `Config` object

In [ ]:
from notebook.base.handlers import APIHandler

class HandlerSidecar(Configurable):
    auth_token = Unicode("auth").tag(config=True)
    
class MyHandler(APIHandler):
    def custom_endpoint(self):
        # self.config is a config object from NotebookWebApp
        sidecar = HandlerSidecar(config=self.config)
        self.auth_token = sidecar.auth_token


## And now back to the slides!
<br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/>


# Finding config files

## `jupyter --paths` shows the directories 

In [ ]:
!jupyter --paths

## JupyterApp defines the config search targets

### `App.name` defines the config file name

**NB**:
`-` is replaced with `_`.

In [ ]:
class SimpleConfig(JupyterApp):
    name="jupyter-simple"
    
    @classmethod
    def show_config_path(cls):
        test = cls()
        for path in test.config_file_paths:
            print(f"{path}/{test.config_file_name}.py")
            print(f"{path}/{test.config_file_name}.json")
            print()


In [ ]:
test = SimpleConfig.show_config_path()

## Examples of this convention in the wild

e.g.,:
- for `jupyter notebook`:  
  `jupyter_notebook_config.py`

- for `jupyter nbconvert`:  
  `jupyter_nbconvert_config.json`

# `jupyter config`:  A new app!

Jupyter config helps you solve lots of config file issues

```
pip install jupyter_config
```

## Find all config files with `jupyter config list`

In [ ]:
!jupyter config list

# Loading config into an app

In [ ]:
class TestConfig(SimpleConfig):
    name="jupyter_test"

    def start(self):
        self.a = A(parent=self)
        self.b = B(parent=self)
        
    def get_started(self):
        # self.initialize loads the config
        # self.start is where you apply app specific logic
        self.initialize([])
        self.start()
        return self

    def _ipython_display_(self):
        display(self.a)
        display(self.b)


In [ ]:
TestConfig.show_config_path()

In [ ]:
TestConfig().get_started()

In [ ]:
%pycat /Users/mpacer/jupyter/config_talk/jupyter_test_config.py

# Specificity wins again!

In [ ]:
class ConflictConfig(TestConfig):
    name = "jupyter_conflict"

ConflictConfig.show_config_path()

In [ ]:
conflict = ConflictConfig().get_started()
display(conflict)

## Find config settings with `jupyter config search`

In [ ]:
!jupyter config search my_trait

# Setting traits from the command line

Trait values are also surfaced from the command line:

```
jupyter notebook --NotebookApp.port=8889 --NotebookApp.open_browser=False
```

Shortened versions are sometimes available via `aliases` and `flags`:

```
jupyter notebook --port=8889 --no-browser
```

## And now back to the slides!
<br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/>


In [ ]:
d